In [116]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [117]:
import pandas as pd
import numpy as np

In [118]:
cwm_df = pd.read_csv('../../data/processed/works/more_works.csv')
cwm_df.head()

,Unnamed: 0,composer,work,work_url
0,0.0,Bizet,Carmen,http://www.wikidata.org/entity/Q185968
1,1.0,Verdi,La traviata,http://www.wikidata.org/entity/Q186162
2,2.0,Verdi,Rigoletto,http://www.wikidata.org/entity/Q189234
3,3.0,Mozart,Don Giovanni,http://www.wikidata.org/entity/Q192039
4,4.0,Puccini,La bohème,http://www.wikidata.org/entity/Q188621


In [133]:
lt_df = pd.read_csv('../../data/processed/listings/operaamerica.csv', index_col=0)
# print(lt_df.shape)
lt_df['composer'] = lt_df.apply(lambda row: row['composer'].replace(u'\xa0', '---'), axis=1)

cw_df = pd.read_csv('../../data/processed/works/wikipedia.csv', index_col=0)
cwm_df = pd.read_csv('../../data/processed/works/more_works.csv', index_col=0)
cwm_df['composer_url'] = '/'
cwm_df['composer_key'] = cwm_df.apply(lambda row: row['composer_url'].split('/')[-1], axis=1)
cwm_df['work_key'] = cwm_df.apply(lambda row: row['work_url'].split('/')[-1], axis=1)
cwm_df = cwm_df.replace(np.nan, '', regex=True)

cwm_df = cwm_df[['composer', 'work', 'composer_url', 'work_url', 'composer_key', 'work_key']]
ref_df = pd.concat([cw_df, cwm_df])
# ref_df.tail(10)

print(lt_df.shape)
lt_df = lt_df.merge(ref_df, left_on=['work', 'composer'], right_on=['work', 'composer'], how='left')
lt_df = lt_df[['theatre', 'work', 'composer', 'composer_key', 'composer_url', 'work_key', 'work_url']]
# lt_df = lt_df.drop_duplicates()
lt_df = lt_df[lt_df['work_key'].isna()]


# lt_df = lt_df.sort_values(by=['work'])
print(lt_df.shape)

lt_df = lt_df.groupby(['work','composer'])['theatre'] \
         .count() \
         .reset_index(name='count') \
         .sort_values(['count'], ascending=False)

lt_df = lt_df.reset_index(drop=True)            
print(lt_df.shape)
lt_df.head(30)            
            

# display(HTML(lt_df.to_html()))


(6830, 7)
(923, 3)


,work,composer,count
0,Concert,---,199
1,Rough Cuts,various,180
2,Sweeney Todd,Sondheim,152
3,A Masked Ball,Verdi,123
4,Carousel,Rodgers,100
5,,---,100
6,Boris Godunov,Mussorgsky,99
7,Julius Caesar,Handel,90
8,The Cunning Little Vixen,Janácek,84
9,Troubadour,Peaslee,83


In [128]:
cwm_df = pd.read_csv('../../data/processed/works/more_works.csv', index_col=0)
cwm_df.head()
# cwm_df[cwm_df['composer']]
cwm_df['composer'].sort_values().unique()
# cwm_df['composer_url'] = '/'
# cwm_df['composer_key'] = cwm_df.apply(lambda row: row['composer_url'].split('/')[-1], axis=1)
# cwm_df['work_key'] = cwm_df.apply(lambda row: row['work_url'].split('/')[-1], axis=1)
# cwm_df = cwm_df.replace(np.nan, '', regex=True)


# ref_df[ref_df['composer']=='Bernstein']

array(['A Chorus Line', 'A Funny Thing Happened on the Way to the Forum',
       'A Little Night Music', 'Acis and Galatea', 'Adamo', 'Adams',
       'Adding Machine', 'Adès', "Ain't Misbehavin'", 'Aldridge',
       'Annie Get Your Gun', 'Antheil', 'Anything Goes', 'Argento',
       'Armide', 'Babes in Toyland', 'Barber', 'Bartók', 'Beethoven',
       'Bellini', 'Benjamin', 'Berg', 'Berio', 'Berlioz', 'Bizet',
       'Blanchard', 'Blitzstein', 'Blow', 'Boito', 'Bolcom', 'Borodin',
       'Braunfels', 'Brigadoon', 'Britten', 'Busoni', 'Cabaret', 'Camelot',
       'Candide', 'Canticle II: Abraham and Isaac', 'Carlson',
       'Carmen Jones', 'Catalani', 'Cato in Utica', 'Catán', 'Cavalli',
       'Cesti', 'Chabrier', 'Charpentier', 'Chin', 'Cilea', 'Cimarosa',
       'Company', 'Copland', 'Corigliano', 'Countess Maritza',
       'Danielpour', 'Daugherty', 'Davies', 'Davis', 'Debussy', 'Delibes',
       'Der Freischutz', 'Der Freischütz',
       'Der Ring des Nibelungen (Ring Cycle)',
   

In [108]:
from bs4 import BeautifulSoup
import requests
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

mappings = []

for idx,row in lt_df.iterrows():
    work = row['work'].strip()
    url = 'https://musicbrainz.org/search?query=' + work.replace(' ', '+') + '&type=work&limit=25'    
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
#     print(url)
    if work != '':
#         print(work)
        try:
            results = soup.find(attrs={'class': 'tbl'}).find('tbody').find_all('tr')[0]
            score = results.find_all('td')[0].get_text()
        except:
            print(url)
        if score=='100':
            name  = results.find_all('td')[1].find('a').get_text()        
            people = results.find_all('td')[2].find('ul').find_all('li')
            for person in people:
                if row['composer'] in person.find('bdi').get_text():                
                    
                    try:
                        work_url  = results.find_all('td')[1].find('a').get('href')
                        work_url  = 'https://musicbrainz.org' + work_url
                        work_r    = requests.get(work_url, headers=headers)
                        work_soup = BeautifulSoup(work_r.text, "html5lib")
                    except:
                        print('Not working!!!')
                        print(work_url)
                        print('Not working!!!')                        
                    try:
                        dbpedia   = work_soup.find(attrs={'class': 'wikidata-favicon'})
                        dbpedia   = 'http:' + dbpedia.find('a').get('href')
                    except:
                        print(row['composer'] + '----' + row['work'])                        
                        print(work_url)
                    
                    
                    
                    mapping = []
                    mapping.append(row['work'])
                    mapping.append(row['composer'])
                    mapping.append(dbpedia)
                    mappings.append(mapping)
#                     mapping.append(person.find('bdi').get_text())        
#                     print(mapping)

https://musicbrainz.org/search?query=Elsewhereless&type=work&limit=25
https://musicbrainz.org/search?query=Tintypes&type=work&limit=25
https://musicbrainz.org/search?query=Acquanetta&type=work&limit=25
https://musicbrainz.org/search?query=Filumena&type=work&limit=25
https://musicbrainz.org/search?query=Szulamit&type=work&limit=25
https://musicbrainz.org/search?query=Dreamkeepers&type=work&limit=25
Mollicone----The Face on the Barroom Floor
https://musicbrainz.org/work/2ba4216f-014c-4530-98d2-49192a233c3d
https://musicbrainz.org/search?query=H2Opus&type=work&limit=25
https://musicbrainz.org/search?query=Rattenbury&type=work&limit=25
https://musicbrainz.org/search?query=Svadba&type=work&limit=25
https://musicbrainz.org/search?query=La+gazzetta&type=work&limit=25
https://musicbrainz.org/search?query=The+Wake+World&type=work&limit=25
https://musicbrainz.org/search?query=IYOV&type=work&limit=25
Lang----The Difficulty of Crossing a Field
https://musicbrainz.org/work/83b0b525-ea1a-4632-ac66-8

KeyboardInterrupt: 

In [131]:
m = [[m[1],m[0],m[2]] for m in mappings]

headers = ['composer', 'work', 'work_url']
df = pd.DataFrame(m, columns=headers)            
# display(HTML(df.to_html()))
df.to_csv('../../data/processed/mapping/ddddddd.csv')

In [130]:
m

[['Bernstein', 'Candide', 'http://www.wikidata.org/wiki/Q1033015'],
 ['Loewe', 'My Fair Lady', 'http://www.wikidata.org/wiki/Q591943'],
 ['Puccini', 'Manon Lescaut', 'http://www.wikidata.org/wiki/Q459341'],
 ['Sondheim', 'A Little Night Music', 'http://www.wikidata.org/wiki/Q2704917'],
 ['Bernstein', 'West Side Story', 'http://www.wikidata.org/wiki/Q669010'],
 ['Rodgers', 'Oklahoma!', 'http://www.wikidata.org/wiki/Q3305977'],
 ['Menotti', 'The Consul', 'http://www.wikidata.org/wiki/Q593511'],
 ['Rodgers', 'South Pacific', 'http://www.wikidata.org/wiki/Q1535240'],
 ['Weill', 'The Threepenny Opera', 'http://www.wikidata.org/wiki/Q212495'],
 ['Leigh', 'Man of La Mancha', 'http://www.wikidata.org/wiki/Q2318125'],
 ['Monteverdi',
  "L'incoronazione di Poppea",
  'http://www.wikidata.org/wiki/Q615522'],
 ['Berlin', 'Annie Get Your Gun', 'http://www.wikidata.org/wiki/Q2419754'],
 ['Rodgers', 'The Sound of Music', 'http://www.wikidata.org/wiki/Q2414348'],
 ['Handel', 'Giulio Cesare', 'http://w

In [ ]:
score = table[0].find_all('td')[0].get_text()
name  = table[0].find_all('td')[1].get_text()
table[0].find_all('td')[2].find('ul').find_all('li')[0].find('bdi').get

In [ ]:
mappings = []

import wikipedia
index=0
for idx,row in lt_df.iterrows():
    index+=1
    if index%100==0:
        print(index)
        
    results = wikipedia.search(row['work'] + ' ' + row['composer'])
#     print('Searching for: ' + row['work_x'])
#     print('Found: ' + str(len(results)))
    if results:
        try:      
            top_result = wikipedia.page(results[0]).title 
            top_result = top_result.replace(' (opera)', '')
            top_result = top_result.replace(' opera', '')
#             print(top_result)
            if top_result.lower() in list(ref_df['work'].str.lower()):
                mapping = []
                work = row['work']
                composer = row['composer']                
                top_result = ref_df[ref_df['work'].str.contains(top_result)]['work_url'].values[0]                
                mapping.append(composer)  
                mapping.append(work)
                mapping.append(top_result)
                print(mapping)
                mappings.append(mapping)
        except:
            None   
            
import pandas as pd                     
headers = ['composer', 'work', 'work_url']
df = pd.DataFrame(mappings, columns=headers)            
df.to_csv('../../data/processed/mapping/ddddddd.csv')
# lt_df = pd.read_csv('../../data/processed/listings/operaamerica.csv', index_col=0)


In [34]:
result.to_csv('../../data/processed/mapping/operaamerica.csv')

In [36]:
# display(HTML(df.to_html()))


In [14]:
df = pd.read_csv('../../data/processed/mapping/operaamerica.csv', index_col=0)

df = df.groupby(['work_x'])['composer_x'] \
         .count() \
         .reset_index(name='count') \
         .sort_values(['count'], ascending=False)

df = df[df['count']>2]            
df            

,work_x,count
23,Alice in Wonderland,6
249,La clemenza di Tito,5
191,Il re pastore,4
155,Griselda,4
401,Semiramide,4
71,Cinderella,4
353,Otello,4
200,Iphigénie en Tauride,4
299,Macbeth,4
508,The Tempest,4


In [64]:
df = pd.read_csv('../../data/processed/works/wikipedia.csv', index_col=0)
# df.head()
df[df['work']=='Cinderella']

,composer,work,composer_url,work_url,composer_key,work_key
2882,Josef Richard Rozkošný,Cinderella,http://www.wikidata.org/entity/Q2332211,http://www.wikidata.org/entity/Q12046368,Q2332211,Q12046368
3722,Peter Maxwell Davies,Cinderella,http://www.wikidata.org/entity/Q139223,http://www.wikidata.org/entity/Q1092280,Q139223,Q1092280
